In [ ]:
!pip install tensorflow-gpu
!pip install --upgrade grpcio
!pip install bert-for-tf2

     |████████████████████████████████| 320.4MB 54kB/s 
Requirement already up-to-date: grpcio in /usr/local/lib/python3.6/dist-packages (1.32.0)
     |████████████████████████████████| 40kB 2.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.6-cp36-none-any.whl size=30318 sha256=f642d52d5e05d7a686bdc366edae9f3e31c722c2dce132a7123777343297af53
  Stored in directory: /root/.cache/pip/wheels/07/a0/b4/75b0601ebaa41e517a797fe9cea119c789664c8408f8a74ae9
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=752cfe8677f4f81faf37e2942da417ced64f4d89da71b32406accd001de5c126
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=45084f2f5a13539a30db106527be6bd50d2a04586c8eb2dc856fd99a8129c02d
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c0

In [ ]:
import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [ ]:
train = pd.read_csv('/content/drive/My Drive/goemotions_aug_dairai_train_cleaned.csv')

In [ ]:
train.head()

,Unnamed: 0,anger,augmented,cleaned_processed,datasource,fear,joy,sadness
0,206509,0.0,0,i feel that some korea guy are handsome and so...,dairai,0.0,1.0,0.0
1,375242,0.0,0,i put my pen to paper and made a list of thing...,dairai,0.0,0.0,0.0
2,166570,1.0,0,i wish i only had to feel the pain of the pett...,dairai,0.0,0.0,0.0
3,200580,0.0,0,i feel passionate about this journey and stand...,dairai,0.0,1.0,0.0
4,300766,0.0,0,i feel like i have convinced myself of these f...,dairai,0.0,1.0,0.0


In [ ]:
train.shape

(460551, 8)

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-10-08 20:08:29--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   153MB/s    in 2.5s    

2020-10-08 20:08:32 (153 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [ ]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
model_dir = './uncased_L-12_H-768_A-12/'
bert_ckpt_file = model_dir + "bert_model.ckpt"
from bert.loader import load_stock_weights
bert_params = bert.params_from_pretrained_ckpt(model_dir)

In [ ]:
classes = ['anger', 'fear', 'joy', 'sadness']

In [ ]:
def get_model(max_seq_len, params):
  input_tensor = keras.layers.Input((max_seq_len, ), dtype='int32')
  bert_layer = bert.BertModelLayer.from_params(params, name='bert')
  bert_output = bert_layer(input_tensor)
  x = keras.layers.Lambda(lambda x: x[:, 0, :])(bert_output)
  x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(768, activation='tanh')(x)
  x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(4, activation='sigmoid')(x)

  model = keras.models.Model(input_tensor, x)
  model.build(input_shape=(None, max_seq_len))
  load_stock_weights(bert_layer, bert_ckpt_file)
  
  
  return model

In [ ]:
from bert.tokenization.bert_tokenization import FullTokenizer
tokenizer = FullTokenizer('uncased_L-12_H-768_A-12/vocab.txt')

In [ ]:
tokens = tokenizer.tokenize('hello bert! hello world!')
print(tokens)
sequence = tokenizer.convert_tokens_to_ids(tokens)
print(sequence)

['hello', 'bert', '!', 'hello', 'world', '!']
[7592, 14324, 999, 7592, 2088, 999]


In [ ]:
train = train.rename(columns={'cleaned_processed':'text'})

In [ ]:
class IntentDetectionData:
  DATA_COLUMN = "text"
  LABEL_COLUMNS = ['anger', 'fear', 'joy', 'sadness']

  def __init__(self, train, tokenizer: FullTokenizer, max_seq_len):
    self.tokenizer = tokenizer
    self.max_seq_len = 0
    self.train_x, self.train_y = self._prepare(train)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x =self._pad(self.train_x)
    self.train_y = self.train_y.astype('int32')
    
  def _prepare(self, df):
    x, y = [], []
    
    for _, row in tqdm(df.iterrows()):
      text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMNS]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      y.append(label)

    return np.array(x), np.array(y)

  def _pad(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [ ]:
train['text_len'] = train['text'].apply(lambda x: len(x.split(' ')))

In [ ]:
train.text_len.sort_values()

308960      1
198631      1
304858      1
164110      1
219996      1
         ... 
110501     80
224855     82
225137    101
78205     102
78768     180
Name: text_len, Length: 460551, dtype: int64

In [ ]:
train = train[train.text_len < 100]

In [ ]:
data = IntentDetectionData(train[:100000], tokenizer, 100)

100000it [01:24, 1188.07it/s]


In [ ]:
with open('train_x_100000.npy', 'wb') as f:
    np.save(f, data.train_x)

In [ ]:
with open('train_y_100000.npy', 'wb') as f:
    np.save(f, data.train_y)

In [ ]:
data.max_seq_len

91

In [ ]:
# model = get_model(data.max_seq_len, bert_params)
model = get_model(data.max_seq_len, bert_params)

Done loading 196 BERT weights from: ./uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fbf414dacc0> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='goemotions_bert_weights_100000.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)
early_stop_callback =  tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2)


In [ ]:
model.compile(keras.optimizers.Adam(5e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(data.train_x, data.train_y, validation_split=0.3, batch_size=32, shuffle=True, epochs=5, callbacks=[model_checkpoint_callback, early_stop_callback])

Epoch 1/5
2188/2188 [==============================] - 1956s 894ms/step - loss: 0.2354 - accuracy: 0.6542 - val_loss: 0.0763 - val_accuracy: 0.8087
Epoch 2/5
2188/2188 [==============================] - 1952s 892ms/step - loss: 0.0699 - accuracy: 0.8138 - val_loss: 0.0587 - val_accuracy: 0.8150
Epoch 3/5
2188/2188 [==============================] - 1953s 893ms/step - loss: 0.0537 - accuracy: 0.8189 - val_loss: 0.0675 - val_accuracy: 0.8400
Epoch 4/5
2188/2188 [==============================] - 1954s 893ms/step - loss: 0.0408 - accuracy: 0.8199 - val_loss: 0.0897 - val_accuracy: 0.8071
Epoch 5/5
2188/2188 [==============================] - 1953s 893ms/step - loss: 0.0342 - accuracy: 0.8250 - val_loss: 0.0967 - val_accuracy: 0.8060


In [ ]:
model.summary()

In [ ]:
model.save_weights('/content/drive/My Drive/goemotion_bert_weights_100000.h5', overwrite=True)

In [ ]:
model = get_model(data.max_seq_len, bert_params)
model.load_weights('/content/drive/My Drive/goemotion_bert_weights_100000.h5')

Done loading 196 BERT weights from: ./uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fbf3596d748> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [ ]:
model.predict(data.train_x[0:10])

array([[2.3150997e-05, 3.3076099e-05, 9.9887902e-01, 6.3355765e-05],
       [2.3511145e-02, 1.3643566e-02, 2.2761660e-02, 3.7019197e-02],
       [9.9998462e-01, 3.8386494e-05, 4.5079380e-04, 7.9335179e-05],
       [1.0806235e-03, 7.5574510e-04, 3.7288973e-01, 1.1009743e-02],
       [1.4841853e-03, 7.0907612e-04, 5.7680482e-01, 9.2981055e-02],
       [2.1988984e-05, 1.0045726e-05, 6.1871469e-05, 9.9992609e-01],
       [2.2278580e-05, 1.0310466e-05, 5.9951872e-05, 9.9992573e-01],
       [2.9060957e-05, 1.8187677e-05, 9.9562621e-01, 7.4902920e-05],
       [5.3904983e-05, 9.9994278e-01, 6.6810178e-05, 3.7717000e-05],
       [2.2593871e-05, 3.1888558e-05, 9.9877602e-01, 5.2689793e-05]],
      dtype=float32)

In [ ]:
data.train_y[0:10]

array([[0, 0, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]], dtype=int32)